In [16]:
import pandas as pd

In [17]:
!pip3 install pandas

In [18]:
patient_info=pd.read_csv('PatientInfoCSVPatient.csv')

In [19]:
patient_info.head()

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,INSURANCE,ETHNICITY
0,249,F,0,116935,Medicare,WHITE
1,249,F,0,149546,Medicare,WHITE
2,249,F,0,158975,Medicare,WHITE
3,250,F,1,124271,Self Pay,BLACK/AFRICAN AMERICAN
4,251,M,0,117937,Private,UNKNOWN/NOT SPECIFIED


In [20]:
pd.get_dummies(patient_info['INSURANCE'])

,Government,Medicaid,Medicare,Private,Self Pay
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,0,0,1
4,0,0,0,1,0
...,...,...,...,...,...
58971,0,0,1,0,0
58972,0,0,0,1,0
58973,0,0,1,0,0
58974,0,0,0,1,0


In [21]:
pd.get_dummies(patient_info['ETHNICITY'])

,AMERICAN INDIAN/ALASKA NATIVE,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,ASIAN,ASIAN - ASIAN INDIAN,ASIAN - CAMBODIAN,ASIAN - CHINESE,ASIAN - FILIPINO,ASIAN - JAPANESE,ASIAN - KOREAN,ASIAN - OTHER,...,PATIENT DECLINED TO ANSWER,PORTUGUESE,SOUTH AMERICAN,UNABLE TO OBTAIN,UNKNOWN/NOT SPECIFIED,WHITE,WHITE - BRAZILIAN,WHITE - EASTERN EUROPEAN,WHITE - OTHER EUROPEAN,WHITE - RUSSIAN
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
58972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
58973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
58974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
note_chunk=pd.read_csv('NOTEEVENTS.csv',chunksize=1)

In [23]:
type(note_chunk.get_chunk(1))

pandas.core.frame.DataFrame

In [24]:
!pip install rank_bm25

In [ ]:
note_chunk=pd.read_csv('NOTEEVENTS.csv',chunksize=1)
cleaned_notes=pd.DataFrame()
c=0

for i in note_chunk:
    c+=1
    #print(c)
    if (c>20000):
        break
    
    cleaned_notes=cleaned_notes.concat(i)
    if (c%100==0):
        print(c)

In [32]:
cleaned_notes['text']=cleaned_notes['text'].apply(clean_text)

KeyError: 'text'

In [ ]:
cleaned_notes.head()

In [ ]:
from nltk.corpus import stopwords
import re
import numpy as np
import re
import string
from stemming.porter2 import stem
def clean_text(note):
    regex = re.compile('[%s]' % re.escape(string.digits))
    note = regex.sub('', note)
    
    note=note.lower()
    
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    note = regex.sub('', note)
    
    sw=stopwords.words('english')
    note = ' '.join([word for word in note.split() if word not in sw])
    
    note = ' '.join([stem(word) for word in note.split()])
    
    note = ' '.join([word for word in note.split() if (len(word)>=4)])
    
    return note

In [ ]:
note_chunk=pd.read_csv('NOTEEVENTS.csv',chunksize=1)

In [ ]:
import random

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
cleaned_notes['Split_Text']=cleaned_notes['text'].apply(lambda x: x.split())
points=[]

In [ ]:
n=20
points=[]
matches=0
tot_match=dict()
tot=dict()
for o in range (1,n):
    
    i=random.randint(1,10000)
    print(i)
    query=cleaned_notes.iloc[i]['TEXT']

    corpus=list(cleaned_notes['TEXT'])

    doc_scores=bm25_compute(query,corpus)

    bests=np.argpartition(doc_scores,-10)[-10:]
    #[a[i] for i in np.argpartition(a,-3)[-3:]]
    a=0;b=0;c=0;d=0;e=0;
    
    insurance_counter=dict()
    
    a1=cleaned_notes.iloc[i]['SUBJECT_ID']
    
    for k in bests:
        
        b1=cleaned_notes.iloc[k]['SUBJECT_ID']
        if(list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0]==list(patients[patients['SUBJECT_ID']==b1]['INSURANCE'])[0]):
            points.append([1,1])
        else:
            points.append([1,0])
        s_id=cleaned_notes.iloc[k]['SUBJECT_ID']
        ins=list(patients[patients['SUBJECT_ID']==s_id]['INSURANCE'])[0]
        if (ins in insurance_counter):
            insurance_counter[ins]+=1
        else:
            insurance_counter[ins]=1
    
    print(insurance_counter)
        #if (k!=i):
            #closest_bm25=k
    
    a1=cleaned_notes.iloc[i]['SUBJECT_ID']
    a2=cleaned_notes.iloc[closest_bm25]['SUBJECT_ID']
    ins2=list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0]
    if (ins2 in tot):
        tot[ins2]+=1
    else:
        tot[ins2]=1
    
    print("Insurance",list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0])
    pred_ins=max(insurance_counter,key=insurance_counter.get)
    ins2=list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0]
    #if(list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0]==list(patients[patients['SUBJECT_ID']==a2]['INSURANCE'])[0]):   
    if(list(patients[patients['SUBJECT_ID']==a1]['INSURANCE'])[0]==pred_ins):
        print("match")
        #if (ins2 in accuracies):
        if ins2 in tot_match:
            tot_match[ins2]+=1
        else:
            tot_match[ins2]=1
        matches+=1
    
print(matches/n)

In [ ]:
np.sum(list(patients['INSURANCE']=='Private'))

In [ ]:
tot_match

In [ ]:
tot

In [ ]:
points

In [ ]:
x=[i[0] for i in points]

In [ ]:
y=[i[1] for i in points]

In [ ]:
data = np.array([x,y])

covMatrix = np.cov(data,bias=True)

In [ ]:
covMatrix

In [ ]:
np.sum(list(patients['INSURANCE']=='Medicare'))

In [ ]:
np.sum(list(patients['INSURANCE']=='Medicaid'))

In [ ]:
np.sum(list(patients['INSURANCE']=='Government'))

In [ ]:
np.sum(list(patients['INSURANCE']=='Private'))

In [ ]:
max(insurance_counter,key=insurance_counter.get)

In [ ]:
matches/n

In [ ]:
np.sort(np.argpartition(doc_scores,-2)[-2:])

In [ ]:
np.argpartition(doc_scores,-2)[-2:]

In [ ]:
a=[4,5,6,2,3,9,14,2,3,4,6,7,9,1]

In [ ]:
np.argpartition(a,-3)[-3:]

In [404]:
[a[i] for i in np.argpartition(a,-3)[-3:]]

[9, 9, 14]

In [237]:
def bm25_compute(query,corpus):
    
    tokenized_corpus = [doc.split(" ") for doc in corpus]

    tokenized_query = query.split(" ")
    bm25 = BM25Okapi(tokenized_corpus)

    doc_scores = bm25.get_scores(tokenized_query)

    #bm25.get_top_n(tokenized_query, corpus, n=1)


    #query = cleaned_notes.iloc[2000]['TEXT']
    
    
    return doc_scores

In [254]:
query='hello'

In [288]:
corpus=list(['hello','hello now it is me','hi again hello'])

In [289]:
print(bm25_compute(query,corpus))

[0.05709406 0.03074296 0.03996584]


In [285]:
temp=list(range(0,1))

In [286]:
a_series = pd. Series(temp)
cleaned_notes['TEMP_COL']=a_series

In [287]:
cleaned_notes['TEMP_COL']

0       0.0
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
9995    NaN
9996    NaN
9997    NaN
9998    NaN
9999    NaN
Name: TEMP_COL, Length: 10000, dtype: float64

In [269]:
cleaned_notes. columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT',
       'Split_Text'],
      dtype='object')

In [257]:
cleaned_notes.size

120000

In [198]:
type(cleaned_notes.iloc[30]['Split_Text'])

list

In [202]:
for i, j in numerator(corpus):
    print(i)

NameError: name 'numerator' is not defined

In [179]:
patients=pd.read_csv('PatientInfoCSVPatient.csv')

In [189]:
patients[patients['SUBJECT_ID']==1000]

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,INSURANCE,ETHNICITY
2786,1000,M,1,143040,Private,UNKNOWN/NOT SPECIFIED


In [190]:
patients[patients['SUBJECT_ID']==1999]

,SUBJECT_ID,GENDER,EXPIRE_FLAG,HADM_ID,INSURANCE,ETHNICITY
3048,1999,M,1,168476,Government,WHITE
3049,1999,M,1,133284,Government,WHITE
3050,1999,M,1,187312,Government,WHITE
